<a href="https://colab.research.google.com/github/Hitika-Jain/LegalTalk/blob/main/vanillabert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Hitika-Jain/LegalTalk.git

%cd LegalTalk
#

Cloning into 'LegalTalk'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 81 (delta 31), reused 24 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 478.32 KiB | 2.75 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/LegalTalk


In [2]:
!pip install -q transformers datasets evaluate scikit-learn accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


debug

In [11]:
import pandas as pd
from collections import Counter
from pathlib import Path
import numpy as np

# update paths if different
TRAIN_CSV = "/content/drive/MyDrive/legal_dataset/train_canonical.csv"
print("Checking file:", TRAIN_CSV)

df = pd.read_csv(TRAIN_CSV, dtype=str)
print("DataFrame columns:", df.columns.tolist())

# 1. check duplicate column names
dups = [c for c,count in Counter(df.columns).items() if count>1]
print("Duplicate column names (if any):", dups)

# 2. If there's a labels column, inspect it
label_col = None
for cand in ['labels_canonical','labels','labels_raw','label','label_vector']:
    if cand in df.columns:
        label_col = cand
        break
print("Using label column:", label_col)

if label_col:
    sample = df[label_col].astype(str).head(10).tolist()
    print("Sample label rows:", sample[:5])

    # try to parse labels into lists (same helper used in pipeline)
    import ast, re
    def parse_label_str(s):
        if pd.isna(s) or str(s).strip()=="":
            return []
        s = str(s).strip()
        if s.startswith("[") and s.endswith("]"):
            try:
                parsed = ast.literal_eval(s)
                if isinstance(parsed, (list,tuple)):
                    return [str(x).strip() for x in parsed if str(x).strip()]
            except Exception:
                pass
        toks = [t.strip() for t in s.replace(",", ";").split(";") if t.strip()]
        return toks

    parsed = [parse_label_str(x) for x in df[label_col].astype(str).head(500)]
    # flatten to get all labels
    all_labels = [lbl for sub in parsed for lbl in sub]
    print("Unique label count (sample 500 rows):", len(set(all_labels)))
    # check duplicates in the full label set if you have 'all_labels' prepared already:
    # If you built all_labels in memory earlier, run this check on that list instead.

# 3. If you already have all_labels in notebook, run:
try:
    all_labels_var = all_labels  # from above
except Exception:
    all_labels_var = None

if all_labels_var is not None:
    from collections import Counter
    c = Counter(all_labels_var)
    dups_labels = [k for k,v in c.items() if v>1]
    print("Duplicate label tokens (in parsed sample):", dups_labels[:20])

Checking file: /content/drive/MyDrive/legal_dataset/train_canonical.csv
DataFrame columns: ['text', 'labels', 'labels_raw', 'labels_canonical']
Duplicate column names (if any): []
Using label column: labels_canonical
Sample label rows: ['IPC_392;IPC_109;IPC_3', 'IPC_436;IPC_5;IPC_494', 'IPC_114;IPC_155;IPC_457;IPC_34', 'IPC_417;IPC_395;IPC_148;IPC_409;IPC_397;IPC_149', 'IPC_306;IPC_326;IPC_336;IPC_332;IPC_379;IPC_323;IPC_324;IPC_313;IPC_468;IPC_320']
Unique label count (sample 500 rows): 97
Duplicate label tokens (in parsed sample): ['IPC_392', 'IPC_109', 'IPC_3', 'IPC_436', 'IPC_5', 'IPC_494', 'IPC_114', 'IPC_155', 'IPC_457', 'IPC_34', 'IPC_417', 'IPC_395', 'IPC_148', 'IPC_409', 'IPC_397', 'IPC_149', 'IPC_306', 'IPC_326', 'IPC_336', 'IPC_332']


In [12]:
# Repair & rebuild label vectors safely
import pandas as pd
from collections import OrderedDict
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import ast, re
from pathlib import Path

IN_CSV = "/content/drive/MyDrive/legal_dataset/train_canonical.csv"
OUT_CSV = "/content/drive/MyDrive/legal_dataset/train_canonical_mlb_fixed.csv"

df = pd.read_csv(IN_CSV, dtype=str).fillna('')
print("Loaded rows:", len(df), "columns:", df.columns.tolist())

# prefer labels_canonical if present
label_col = 'labels_canonical' if 'labels_canonical' in df.columns else ('labels' if 'labels' in df.columns else None)
if label_col is None:
    raise ValueError("No labels column found in CSV.")

# parse label strings into list
def parse_label_str(s):
    if pd.isna(s) or str(s).strip() == "":
        return []
    s = str(s).strip()
    if s.startswith("[") and s.endswith("]"):
        try:
            parsed = ast.literal_eval(s)
            if isinstance(parsed, (list,tuple)):
                return [str(x).strip() for x in parsed if str(x).strip()]
        except Exception:
            pass
    toks = [t.strip() for t in re.split(r'[;,]\s*|\s+', s) if t.strip()]
    return toks

df['labels_list'] = df[label_col].apply(parse_label_str)

# build ordered unique label list (preserving first occurrence)
seen = OrderedDict()
for lbls in df['labels_list']:
    for l in lbls:
        if l not in seen:
            seen[l] = True
all_labels = list(seen.keys())
print("Constructed ordered unique label list. Count =", len(all_labels))

# sanity: ensure no duplicates
assert len(all_labels) == len(set(all_labels)), "still duplicates in all_labels!"

# Build MLb with explicit classes and fit empty to set classes safely
mlb = MultiLabelBinarizer(classes=all_labels)
mlb.fit([])  # OK since classes already provided

# transform label lists -> vectors
label_vectors = mlb.transform(df['labels_list'])
df['label_vector'] = list(label_vectors.astype(float))

# Save fixed CSV for reproducibility
df.to_csv(OUT_CSV, index=False)
print("Wrote fixed CSV:", OUT_CSV)

# Build HF Dataset to sanity-check
text_col = 'text' if 'text' in df.columns else df.columns[0]
hf_ds = Dataset.from_dict({
    "text": df[text_col].tolist(),
    "labels": [list(map(float, v)) for v in df['label_vector'].tolist()]
})
print("HF Dataset rows:", len(hf_ds))
# keep mlb for later
import pickle
with open(Path(OUT_CSV).parent / "mlb_fixed.pkl", "wb") as f:
    pickle.dump(mlb, f)
print("Saved mlb_fixed.pkl next to CSV.")

Loaded rows: 42750 columns: ['text', 'labels', 'labels_raw', 'labels_canonical']
Constructed ordered unique label list. Count = 97
Wrote fixed CSV: /content/drive/MyDrive/legal_dataset/train_canonical_mlb_fixed.csv
HF Dataset rows: 42750
Saved mlb_fixed.pkl next to CSV.


actual model

In [6]:
# Install deps (if not already)
!pip install -q datasets transformers

# Memory-efficient preprocessing + tokenization (HuggingFace datasets + arrow)
import os, ast, re, math
from pathlib import Path
from collections import OrderedDict, Counter
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, disable_caching
from transformers import AutoTokenizer

# ========== CONFIG =============
TRAIN_CSV = "/content/drive/MyDrive/legal_dataset/train_canonical_mlb_fixed.csv"
DEV_CSV   = "/content/drive/MyDrive/legal_dataset/dev_canonical.csv"   # optional
OUTPUT_ARROW_DIR = "/content/arrow_dsets"   # where arrow cache will go
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
MAX_LEN = 256
TOKENIZE_BATCH_SIZE = 256   # batch size for .map during tokenization (reduce if OOM)
# ===============================

Path(OUTPUT_ARROW_DIR).mkdir(parents=True, exist_ok=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# --- 1) build label vocabulary w/ low memory by scanning only the 'labels' column in chunks ---
def parse_label_str(s):
    if pd.isna(s) or str(s).strip()=="":
        return []
    s = str(s).strip()
    if s.startswith("[") and s.endswith("]"):
        try:
            parsed = ast.literal_eval(s)
            if isinstance(parsed, (list,tuple)):
                return [str(x).strip() for x in parsed if str(x).strip()]
        except Exception:
            pass
    toks = [t.strip() for t in re.split(r'[;,]\s*|\s+', s) if t.strip()]
    return toks

print("Scanning labels column to build vocabulary (low memory)...")
label_ordered = OrderedDict()
for chunk in pd.read_csv(TRAIN_CSV, usecols=['labels','labels_canonical'], dtype=str, chunksize=20000, iterator=True):
    # prefer labels_canonical if present
    col = 'labels_canonical' if 'labels_canonical' in chunk.columns else 'labels'
    for s in chunk[col].astype(str).head(len(chunk)):
        for lab in parse_label_str(s):
            if lab not in label_ordered:
                label_ordered[lab] = True
all_labels = list(label_ordered.keys())
print("Found labels:", len(all_labels))

# Save label list for reproducibility
with open(Path(OUTPUT_ARROW_DIR)/"label_list.txt","w",encoding="utf-8") as f:
    for l in all_labels: f.write(l+"\n")

# Build mapping label->index (for fast vector creation)
label2idx = {lab:i for i,lab in enumerate(all_labels)}
num_labels = len(all_labels)
print("Num labels:", num_labels)

# --- 2) load CSVs into HuggingFace datasets (disk-backed Arrow) ---
# load_dataset will create arrow files on disk (memory-efficient)
data_files = {"train": TRAIN_CSV}
if Path(DEV_CSV).exists():
    data_files["validation"] = DEV_CSV

print("Loading CSV into datasets (arrow). This does NOT tokenize yet.")
ds = load_dataset("csv", data_files=data_files, cache_dir=OUTPUT_ARROW_DIR)
print(ds)

# --- 3) define functions to parse labels -> list of ints (indices) and to tokenize ---
def labels_to_multihot(labelstrs):
    # labelstrs: single string like "IPC_302;IPC_120B"
    toks = parse_label_str(labelstrs)
    # create multihot list of floats length num_labels
    vec = [0]*num_labels
    for t in toks:
        idx = label2idx.get(t)
        if idx is not None:
            vec[idx] = 1
        else:
            # token not found in vocabulary: ignore or log (we ignore here)
            pass
    return vec

def tokenize_and_encode(batch):
    # batch is dict of lists
    texts = batch["text"] if "text" in batch else batch.get("sentence_text")
    # tokenize (batched)
    toks = tokenizer(texts, padding="max_length", truncation=True, max_length=MAX_LEN)
    # create label vectors
    labels = [labels_to_multihot(s) for s in batch.get("labels_canonical", batch.get("labels"))]
    # Convert labels to float type explicitly
    toks["labels"] = [list(map(float, l)) for l in labels]
    return toks

# --- 4) Map tokenization + label vector creation in batches, write to arrow (disk), free memory ---
print("Tokenizing and encoding into Arrow format (batched). This writes to disk and uses little RAM.")
map_kwargs = {"batched": True, "batch_size": TOKENIZE_BATCH_SIZE, "remove_columns": ds["train"].column_names, "num_proc": 1}
ds_tokenized = ds.map(tokenize_and_encode, **map_kwargs, load_from_cache_file=True)
# Note: load_from_cache_file=True will reuse existing mapping if available; set False to force re-run.
print(ds_tokenized)

# --- 5) set format for PyTorch (no big Python lists kept in memory) ---
ds_tokenized.set_format(type="torch", columns=["input_ids","attention_mask","labels"])
print("Done. Tokenized datasets ready for Trainer. Arrow cache directory:", OUTPUT_ARROW_DIR)

Scanning labels column to build vocabulary (low memory)...
Found labels: 97
Num labels: 97
Loading CSV into datasets (arrow). This does NOT tokenize yet.
DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'labels_raw', 'labels_canonical', 'labels_list', 'label_vector'],
        num_rows: 42750
    })
    validation: Dataset({
        features: ['text', 'labels', 'labels_raw', 'labels_canonical', 'labels_list', 'label_vector'],
        num_rows: 10181
    })
})
Tokenizing and encoding into Arrow format (batched). This writes to disk and uses little RAM.


Map:   0%|          | 0/42750 [00:00<?, ? examples/s]

Map:   0%|          | 0/10181 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 42750
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10181
    })
})
Done. Tokenized datasets ready for Trainer. Arrow cache directory: /content/arrow_dsets


In [7]:
# ========== TRAINER / TRAIN / EVAL (run after tokenization) ==========
import os, json, numpy as np
from pathlib import Path
import torch # Import torch library
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    AutoTokenizer
)
from sklearn.metrics import f1_score, precision_recall_fscore_support
import pickle

# ---------- CONFIG (edit if needed) ----------
OUTPUT_ARROW_DIR = "/content/arrow_dsets"   # same as tokenization output_dir
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
OUTPUT_DIR = "/content/legalbert_gold_run"   # where final model + artifacts will be stored
NUM_EPOCHS = 3
TRAIN_BATCH = 8
EVAL_BATCH = 8
LR = 2e-5
WEIGHT_DECAY = 0.01
EVAL_STEPS = 500
SAVE_STEPS = 500
MAX_LEN = 256
SEED = 42
# -------------------------------------------------------------------

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# 1) Load tokenized datasets from ds_tokenized (created earlier)
# Expect ds_tokenized to be the returned object from `datasets.map` in the tokenization step.
# It usually looks like: {'train': Dataset, 'validation': Dataset}
try:
    train_dataset = ds_tokenized["train"]
except Exception as e:
    raise RuntimeError("ds_tokenized not found in the notebook. Run the tokenization cell first.") from e

val_dataset = ds_tokenized.get("validation", None)

# 2) Load label list and label2idx mapping saved earlier
label_list_path = Path(OUTPUT_ARROW_DIR) / "label_list.txt"
if not label_list_path.exists():
    # if label_list not present, try to reconstruct from dataset 'labels' length
    raise FileNotFoundError(f"label_list.txt not found in {OUTPUT_ARROW_DIR}. Create or point to it.")
with open(label_list_path, "r", encoding="utf-8") as f:
    label_list = [l.strip() for l in f if l.strip()]
num_labels = len(label_list)
label2idx = {lab:i for i,lab in enumerate(label_list)}
idx2label = {i:lab for lab,i in label2idx.items()}

print("Loaded", num_labels, "labels. Sample:", label_list[:20])

# 3) Load tokenizer + model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
config = AutoConfig.from_pretrained(MODEL_NAME, num_labels=num_labels)
config.problem_type = "multi_label_classification"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config)

# 4) Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 5) Metrics function (sigmoid -> threshold)
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= 0.5).astype(int)
    micro = f1_score(labels, preds, average='micro', zero_division=0)
    macro = f1_score(labels, preds, average='macro', zero_division=0)
    return {"f1_micro": micro, "f1_macro": macro}

# 6) Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH,
    per_device_eval_batch_size=EVAL_BATCH,
    learning_rate=LR,
    weight_decay=WEIGHT_DECAY,
    eval_strategy="steps" if val_dataset is not None else "no", # Changed from evaluation_strategy
    eval_steps=EVAL_STEPS,
    save_steps=SAVE_STEPS,
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True if val_dataset is not None else False,
    metric_for_best_model="f1_micro",
    fp16=True if ( (torch.cuda.is_available()) and (torch.cuda.get_device_properties(0).total_memory > 8000000000) ) else False,
    seed=SEED,
)

# 7) Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics if val_dataset is not None else None,
)

# 8) Train
train_result = trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Save label mapping for inference
with open(Path(OUTPUT_DIR)/"label_list.json","w",encoding="utf-8") as f:
    json.dump(label_list, f, ensure_ascii=False, indent=2)
with open(Path(OUTPUT_DIR)/"label2idx.pkl","wb") as f:
    pickle.dump(label2idx, f)

print("Training complete. Artifacts saved to", OUTPUT_DIR)

# 9) Evaluate & threshold tuning (if validation set exists)
if val_dataset is not None:
    print("Running evaluation and threshold tuning on validation set...")
    preds_out = trainer.predict(val_dataset)
    logits = preds_out.predictions
    labels = preds_out.label_ids
    probs = 1 / (1 + np.exp(-logits))

    # basic metrics @0.5
    preds_05 = (probs >= 0.5).astype(int)
    micro05 = f1_score(labels, preds_05, average='micro', zero_division=0)
    print("Validation micro-F1 @0.5:", micro05)

    # global threshold tuning (coarse)
    best_t = 0.5
    best_f1 = micro05
    for t in np.linspace(0.1, 0.9, 17):
        p = (probs >= t).astype(int)
        f = f1_score(labels, p, average='micro', zero_division=0)
        if f > best_f1:
            best_f1 = f; best_t = t
    print("Best global threshold:", best_t, "micro-F1:", best_f1)

    # simple per-label threshold tuning: find best threshold per label by F1 on validation
    # (this is slow if many labels; sample only for labels with >N positives)
    from sklearn.metrics import f1_score
    per_label_thresholds = []
    for i in range(num_labels):
        y_true = labels[:, i]
        y_prob = probs[:, i]
        if y_true.sum() < 5:
            per_label_thresholds.append(0.5)  # not enough positives; keep 0.5
            continue
        # grid search for best threshold for this label
        best_t_i = 0.5
        best_f_i = 0.0
        for t in np.linspace(0.1, 0.9, 17):
            y_pred = (y_prob >= t).astype(int)
            f_i = f1_score(y_true, y_pred, zero_division=0)
            if f_i > best_f_i:
                best_f_i = f_i
                best_t_i = t
        per_label_thresholds.append(best_t_i)

    # save thresholds
    np.save(Path(OUTPUT_DIR)/"per_label_thresholds.npy", np.array(per_label_thresholds))
    print("Saved per-label thresholds (n={})".format(len(per_label_thresholds)))

    # Save dev probs/labels for later analyses
    np.save(Path(OUTPUT_DIR)/"val_probs.npy", probs)
    np.save(Path(OUTPUT_DIR)/"val_labels.npy", labels)

print("Done training+eval pipeline.")

Loaded 97 labels. Sample: ['IPC_392', 'IPC_109', 'IPC_3', 'IPC_436', 'IPC_5', 'IPC_494', 'IPC_114', 'IPC_155', 'IPC_457', 'IPC_34', 'IPC_417', 'IPC_395', 'IPC_148', 'IPC_409', 'IPC_397', 'IPC_149', 'IPC_306', 'IPC_326', 'IPC_336', 'IPC_332']


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2204893377.py:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss,F1 Micro,F1 Macro
500,0.141500,0.147892,0.000000,0.000000
1000,0.141000,0.145460,0.000000,0.000000
1500,0.134900,0.140895,0.091824,0.007129
2000,0.133700,0.135806,0.111237,0.009689
2500,0.126500,0.131563,0.111490,0.012313
3000,0.124200,0.128640,0.141465,0.023186
3500,0.121300,0.126304,0.135978,0.019925
4000,0.118200,0.124352,0.160393,0.025667
4500,0.119400,0.123223,0.173513,0.032434
5000,0.116800,0.121892,0.162173,0.026708


Training complete. Artifacts saved to /content/legalbert_gold_run
Running evaluation and threshold tuning on validation set...


Validation micro-F1 @0.5: 0.30199396945822393
Best global threshold: 0.25 micro-F1: 0.43417352625389843
Saved per-label thresholds (n=97)
Done training+eval pipeline.


In [11]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, classification_report
import pickle
from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer

# Define OUTPUT_DIR and OUTPUT_ARROW_DIR
OUTPUT_DIR = "/content/legalbert_gold_run"
OUTPUT_ARROW_DIR = "/content/arrow_dsets" # Path where label_list.txt was saved

# 1. Load label list and reconstruct MultiLabelBinarizer
label_list_path = Path(OUTPUT_ARROW_DIR) / "label_list.txt"
if not label_list_path.exists():
    raise FileNotFoundError(f"label_list.txt not found in {OUTPUT_ARROW_DIR}. Ensure tokenization was run.")

with open(label_list_path, "r", encoding="utf-8") as f:
    label_list = [l.strip() for l in f if l.strip()]

mlb = MultiLabelBinarizer(classes=label_list)
mlb.fit([]) # Fit with empty data to set the classes

# ---------------------------
# 2. Load test dataset
# ---------------------------
test_df = pd.read_csv("/content/drive/MyDrive/legal_dataset/test_canonical.csv")

# Parse canonical labels → lists
test_df["labels"] = test_df["labels_canonical"].apply(
    lambda x: x.split(";") if isinstance(x, str) else []
)

# Convert to one-hot using the SAME MultiLabelBinarizer used for training
y_test = mlb.transform(test_df["labels"])
print("Test set size:", len(test_df))

# ---------------------------
# 3. Tokenize test samples
# ---------------------------
def tokenize_batch(texts, max_len=256):
    return tokenizer(
        texts, padding=True, truncation=True, max_length=max_len, return_tensors="pt"
    )

# ---------------------------
# 4. Get model predictions
# ---------------------------
model.eval()
all_preds = []

batch_size = 16
for i in tqdm(range(0, len(test_df), batch_size)):
    batch_texts = test_df["text"].iloc[i : i + batch_size].tolist()

    enc = tokenize_batch(batch_texts)
    enc = {k: v.to(model.device) for k, v in enc.items()}

    with torch.no_grad():
        logits = model(**enc).logits
        probs = torch.sigmoid(logits).cpu().numpy()

    all_preds.append(probs)

all_preds = np.vstack(all_preds)
print("Predictions shape:", all_preds.shape)

# ---------------------------
# 5. Apply thresholds
# ---------------------------

# If per-label thresholds exist
try:
    per_label_thr = np.load(Path(OUTPUT_DIR)/"per_label_thresholds.npy")
    print("Using per-label thresholds.")
    y_pred = (all_preds >= per_label_thr).astype(int)
except FileNotFoundError:
    # Use global threshold if per-label thresholds not found
    # The 'best_t' from training is a good candidate for 'best_threshold_global'
    # If 'best_t' is not available, a default of 0.5 can be used.
    # For now, let's assume 'best_t' is available from the training cell if it ran successfully.
    # If not, you might need to hardcode a default or re-run the training and evaluation.
    # Given 'best_t' is in the kernel state, we can use it.
    print(f"Per-label thresholds not found. Using best global threshold: {best_t}")
    y_pred = (all_preds >= best_t).astype(int)
except NameError:
    # Fallback if best_t is not defined (e.g., if evaluation part of training was skipped)
    print("Neither per-label thresholds nor best_t found. Using default global threshold: 0.5")
    y_pred = (all_preds >= 0.5).astype(int)

# ---------------------------
# 6. Compute metrics
# ---------------------------
micro_f1 = f1_score(y_test, y_pred, average="micro")
macro_f1 = f1_score(y_test, y_pred, average="macro")

print("\n===== TEST PERFORMANCE =====")
print("Micro-F1:", micro_f1)
print("Macro-F1:", macro_f1)

# Optional: per-label F1
report = classification_report(
    y_test, y_pred, target_names=mlb.classes_, zero_division=0
)
print(report)

# Save predictions
out_df = test_df.copy()
out_df["predicted_labels"] = [
    ";".join(np.array(mlb.classes_)[row == 1]) for row in y_pred
]

out_df.to_csv("/content/legalbert_test_predictions.csv", index=False)
print("Saved → legalbert_test_predictions.csv")

Test set size: 13019


100%|██████████| 814/814 [01:45<00:00,  7.69it/s]


Predictions shape: (13019, 97)
Using per-label thresholds.

===== TEST PERFORMANCE =====
Micro-F1: 0.4136242894572224
Macro-F1: 0.24397265959041137
              precision    recall  f1-score   support

     IPC_392       0.19      0.33      0.24       202
     IPC_109       0.12      0.04      0.06       256
       IPC_3       0.00      0.00      0.00       141
     IPC_436       0.75      0.48      0.59       442
       IPC_5       0.48      0.74      0.58      3731
     IPC_494       0.70      0.59      0.64      1200
     IPC_114       0.41      0.39      0.40      1293
     IPC_155       0.04      0.01      0.01       261
     IPC_457       0.33      0.39      0.36       525
      IPC_34       0.76      0.72      0.73      3462
     IPC_417       0.48      0.63      0.55      1262
     IPC_395       0.36      0.50      0.42       720
     IPC_148       0.00      0.00      0.00       107
     IPC_409       0.03      0.01      0.01       100
     IPC_397       0.18      0.48      0.

model 2

In [3]:
# LegalBERT training (gold-only). Copy into one Colab cell and run.
import os, pickle, math
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_recall_fscore_support
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)

# ------------- CONFIG -------------
TRAIN_CSV = "/content/drive/MyDrive/legal_dataset/train_canonical_mlb_fixed.csv"
DEV_CSV   = "/content/drive/MyDrive/legal_dataset/dev_canonical.csv"   # optional but recommended
TEST_CSV  = "/content/drive/MyDrive/legal_dataset/test_canonical.csv"                                          # optional
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
OUTPUT_DIR = "/content/legalbert_gold_run"
MAX_LEN = 256
BATCH_SIZE = 8
EPOCHS = 3
LR = 2e-5
WEIGHT_DECAY = 0.01
EVAL_STEPS = 500
SEED = 42
# ----------------------------------

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# 1) Load CSVs
def load_csv(path):
    df = pd.read_csv(path, dtype=str).fillna('')
    # accept either 'labels_canonical' (preferred) or 'labels' column
    if 'labels_canonical' in df.columns:
        df = df.rename(columns={'labels_canonical': 'labels'})
    if 'text' not in df.columns:
        # try common alternatives
        for c in ['sentence_text','sentence','document_text','content']:
            if c in df.columns:
                df = df.rename(columns={c: 'text'})
                break
    if 'text' not in df.columns or 'labels' not in df.columns:
        raise ValueError(f"CSV {path} must contain 'text' and 'labels' (semicolon separated canonical IDs)")

    # Explicitly create a new DataFrame with only the required columns and a pristine, unique RangeIndex
    return pd.DataFrame({'text': df['text'].values.tolist(), 'labels': df['labels'].values.tolist()}).reset_index(drop=True)

train_df = load_csv(TRAIN_CSV)
dev_df = load_csv(DEV_CSV) if DEV_CSV else None
test_df = load_csv(TEST_CSV) if TEST_CSV else None

# 2) Parse label strings into lists
def parse_label_str(s):
    s_stripped = str(s).strip()
    if s_stripped == "":
        return []
    # handle semicolon or comma separated canonical tokens or python list
    if s_stripped.startswith("[") and s_stripped.endswith("]"):
        try:
            import ast
            parsed = ast.literal_eval(s_stripped)
            if isinstance(parsed, (list,tuple)):
                return [str(x).strip() for x in parsed if str(x).strip()]
        except Exception:
            pass
    # split
    toks = [t.strip() for t in s_stripped.replace(",", ";").split(";") if t.strip()]
    return toks

train_df['labels_list'] = train_df['labels'].apply(parse_label_str)
if dev_df is not None:
    dev_df['labels_list'] = dev_df['labels'].apply(parse_label_str)
if test_df is not None:
    test_df['labels_list'] = test_df['labels'].apply(parse_label_str)

In [ ]:
# 3) Build label vocabulary (from training only)
all_labels = sorted({lbl for row in train_df['labels_list'] for lbl in row})
print("Num labels:", len(all_labels))
print("Sample labels:", all_labels[:20])

mlb = MultiLabelBinarizer(classes=all_labels)
y_train = mlb.fit_transform(train_df['labels_list'])

# attach binary vectors
train_df['label_vector'] = list(y_train.astype(float))
if dev_df is not None:
    dev_df['label_vector'] = list(mlb.transform(dev_df['labels_list']).astype(float))
if test_df is not None:
    test_df['label_vector'] = list(mlb.transform(test_df['labels_list']).astype(float))

# 4) Tokenizer & dataset preparation
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def make_hf_dataset(df):
    ds = Dataset.from_dict({'text': df['text'].tolist(), 'labels': df['label_vector'].tolist()})
    def fn(examples):
        toks = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=MAX_LEN)
        toks['labels'] = [list(map(float, l)) for l in examples['labels']]
        return toks
    tokenized = ds.map(fn, batched=True, remove_columns=['text','labels'])
    tokenized.set_format(type='torch')
    return tokenized

train_ds = make_hf_dataset(train_df)
dev_ds = make_hf_dataset(dev_df) if dev_df is not None else None
test_ds = make_hf_dataset(test_df) if test_df is not None else None

Num labels: 42972
Sample labels: ['0', '0;Section 107;Section 5;Section 324', '0;Section 109;Section 34;Section 395;Section 417;Section 415', '0;Section 114;Section 500', '0;Section 120;Section 147;Section 120B;Section 353;Section 13', '0;Section 13;Section 109;Section 500;Section 2;Section 5;Section 34;Section 427', '0;Section 13;Section 500;Section 304B;Section 120B;Section 147;Section 313;Section 419;Section 394;Section 392;Section 34;Section 498A', '0;Section 149;Section 304B;Section 34;Section 380;Section 354;Section 313;Section 193', '0;Section 155;Section 114', '0;Section 161;Section 467;Section 427;Section 457;Section 468;Section 107;Section 397;Section 465', '0;Section 186;Section 468;Section 148', '0;Section 2', '0;Section 228;Section 302;Section 300', '0;Section 2;Section 13;Section 448;Section 120B', '0;Section 2;Section 336;Section 395;Section 313;Section 417;Section 120B', '0;Section 2;Section 500', '0;Section 300;Section 332;Section 228', '0;Section 300;Section 494;Secti

In [ ]:
# 5) Model (multi-label)
config = AutoConfig.from_pretrained(MODEL_NAME, num_labels=len(all_labels))
config.problem_type = "multi_label_classification"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config)

# 6) Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= 0.5).astype(int)
    micro = f1_score(labels, preds, average='micro', zero_division=0)
    macro = f1_score(labels, preds, average='macro', zero_division=0)
    return {"f1_micro": micro, "f1_macro": macro}

In [ ]:
# 7) Trainer
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    weight_decay=WEIGHT_DECAY,
    evaluation_strategy="steps" if dev_ds else "no",
    eval_steps=EVAL_STEPS,
    logging_steps=100,
    save_steps=EVAL_STEPS,
    save_total_limit=2,
    load_best_model_at_end=True if dev_ds else False,
    metric_for_best_model="f1_micro",
    fp16=torch.cuda.is_available()
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics if dev_ds else None
)

# 8) Train
trainer.train()

# 9) Save model & mlb
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
with open(Path(OUTPUT_DIR)/"mlb.pkl","wb") as f:
    pickle.dump(mlb, f)

print("Saved model, tokenizer, mlb to", OUTPUT_DIR)

# 10) Evaluate on dev/test and threshold tuning
def predict_and_eval(dataset, name="dev"):
    out = trainer.predict(dataset)
    logits = out.predictions
    probs = 1 / (1 + np.exp(-logits))
    labels = out.label_ids
    # default 0.5
    preds0 = (probs >= 0.5).astype(int)
    micro0 = f1_score(labels, preds0, average='micro', zero_division=0)
    print(f"{name} micro F1 @0.5 = {micro0:.4f}")
    # quick threshold tuning (global threshold)
    best_t, best_f1 = 0.5, micro0
    for t in np.linspace(0.1, 0.9, 17):
        preds_t = (probs >= t).astype(int)
        f = f1_score(labels, preds_t, average='micro', zero_division=0)
        if f > best_f1:
            best_f1 = f; best_t = t
    print(f"{name} best global-threshold tuning -> t={best_t:.2f}, micro_f1={best_f1:.4f}")
    return probs, labels

if dev_ds:
    dev_probs, dev_labels = predict_and_eval(dev_ds, "dev")
if test_ds:
    test_probs, test_labels = predict_and_eval(test_ds, "test")

# Save dev probs optionally for threshold tuning later
if dev_ds:
    np.save(Path(OUTPUT_DIR)/"dev_probs.npy", dev_probs)
    np.save(Path(OUTPUT_DIR)/"dev_labels.npy", dev_labels)

model 3

In [ ]:
# Ensemble pipeline: candidate generation -> meta-classifier
# Copy-paste this into Colab and run.
# Requires: sentence-transformers, faiss-cpu, transformers, datasets, lightgbm, scikit-learn, pandas

!pip install -q sentence-transformers faiss-cpu transformers datasets lightgbm scikit-learn

import os, json, time, math, random
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd

# Sentence Transformer for retrieval & statute embeddings
from sentence_transformers import SentenceTransformer, util
import faiss

# Huggingface Transformers for LegalBERT inference
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# LightGBM for meta model
import lightgbm as lgb
from sklearn.metrics import f1_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split

# ----------------- CONFIG -----------------
# Data (these CSVs must have columns: text, labels_canonical (semicolon-separated canonical IDs))
TRAIN_CSV = "/content/drive/MyDrive/legal_dataset/train_canonical_mlb_fixed.csv"
DEV_CSV   = "/content/drive/MyDrive/legal_dataset/dev_canonical.csv"
TEST_CSV  = "/content/drive/MyDrive/legal_dataset/test_canonical.csv"   # if you have one

# Statutes file (must include a canonical id column like 'IPC_302' and description columns)
STATUTES_CSV = "/content/drive/MyDrive/legal_dataset/statutes.csv"   # update if needed

# Pretrained models
SBERT_NAME = "sentence-transformers/all-MiniLM-L6-v2"    # fast bi-encoder
LEGALBERT_DIR = "/content/legalbert_gold_run3"            # directory with saved LegalBERT (from previous training)

# Candidate gen & training params
TOPK_RETRIEVE = 10   # top-K from SBERT retrieval
TOPK_NEIGHBORS = 5   # for statute-neighbor expansion (on statute-embeddings)
NEGATIVE_SAMPLE_PER_POS = 2  # number of negatives to sample per positive candidate for meta training

# Training meta-model
META_RANDOM_STATE = 42
LIGHTGBM_PARAMS = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "n_jobs": 4,
    "seed": META_RANDOM_STATE
}

OUT_DIR = Path("/content/ensemble_out")
OUT_DIR.mkdir(exist_ok=True, parents=True)
# ------------------------------------------

# ----------------- UTILS & LOADERS -----------------
def read_canonical_labels(s):
    if pd.isna(s) or str(s).strip()=="":
        return []
    return [x.strip() for x in str(s).split(";") if x.strip()]

# load datasets (stream-friendly)
train_df = pd.read_csv(TRAIN_CSV, dtype=str).fillna('')
dev_df   = pd.read_csv(DEV_CSV, dtype=str).fillna('') if Path(DEV_CSV).exists() else None
test_df  = pd.read_csv(TEST_CSV, dtype=str).fillna('') if Path(TEST_CSV).exists() else None

for df,name in [(train_df,"train"),(dev_df,"dev"),(test_df,"test")]:
    if df is None: continue
    if 'text' not in df.columns:
        raise ValueError(f"{name} CSV must contain 'text' column.")
    if 'labels_canonical' not in df.columns and 'labels' not in df.columns:
        raise ValueError(f"{name} CSV must contain 'labels_canonical' or 'labels' column.")
    # add parsed list
    col = 'labels_canonical' if 'labels_canonical' in df.columns else 'labels'
    df['labels_list'] = df[col].apply(read_canonical_labels)

print("Loaded datasets: train", len(train_df), "dev", len(dev_df) if dev_df is not None else None, "test", len(test_df) if test_df is not None else None)

# ----------------- Build statute definitions & list -----------------
stat_df = pd.read_csv(STATUTES_CSV, dtype=str).fillna('')
# find canonical id column
id_col = None
for c in stat_df.columns:
    if c.lower() in ('id','statute_id','canonical_id','code'):
        id_col = c; break
if id_col is None:
    id_col = stat_df.columns[0]

# Compose a reasonable definition text per statute by concatenating helpful columns:
desc_cols = [c for c in ['description','offense','punishment','def','text'] if c in stat_df.columns]
if not desc_cols:
    # fallback: use first non-id column
    desc_cols = [c for c in stat_df.columns if c != id_col][:1]

stat_df['definition'] = stat_df[desc_cols].astype(str).agg(' '.join, axis=1).str.replace(r'\s+',' ', regex=True).str.strip()
statute_ids = stat_df[id_col].tolist()
statute_defs = stat_df['definition'].tolist()
print("Statutes loaded:", len(statute_ids))

# compute statute prior probabilities from train set
from collections import Counter
counter = Counter()
for labs in train_df['labels_list']:
    counter.update(labs)
total_label_appearances = sum(counter.values()) if counter else 1
prior_map = {s: counter.get(s,0)/total_label_appearances for s in statute_ids}

# ----------------- SBERT: build statute embeddings & FAISS  -----------------
print("Loading SBERT:", SBERT_NAME)
sbert = SentenceTransformer(SBERT_NAME)

STAT_EMB_NPY = OUT_DIR/"statute_emb.npz"
if STAT_EMB_NPY.exists():
    arr = np.load(STAT_EMB_NPY, allow_pickle=True)
    statute_embeddings = arr['emb']
    print("Loaded statute embeddings from", STAT_EMB_NPY)
else:
    print("Encoding statute definitions with SBERT... (this may take a minute)")
    statute_embeddings = sbert.encode(statute_defs, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
    np.savez_compressed(STAT_EMB_NPY, ids=np.array(statute_ids), defs=np.array(statute_defs), emb=statute_embeddings)
    print("Saved statute embeddings to", STAT_EMB_NPY)

# faiss index
dim = statute_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(statute_embeddings.astype('float32'))

# ----------------- helper: retrieve & neighbors -----------------
def retrieve_candidates_by_text(text, topk=TOPK_RETRIEVE):
    q_emb = sbert.encode([text], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q_emb.astype('float32'), topk)
    results = []
    for score, idx in zip(D[0], I[0]):
        results.append((statute_ids[idx], statute_defs[idx], float(score)))
    return results

def statute_neighbors(stat_id, topk=TOPK_NEIGHBORS):
    # find index
    try:
        idx = statute_ids.index(stat_id)
    except ValueError:
        return []
    D, I = index.search(statute_embeddings[idx:idx+1].astype('float32'), topk+1)
    out = []
    for j, idx2 in enumerate(I[0]):
        if statute_ids[idx2] == stat_id: continue
        out.append((statute_ids[idx2], statute_defs[idx2], float(D[0][j])))
    return out[:topk]

# ----------------- simple regex extractor -----------------
import re
# a concise regex: supports "Section 302", "section 302", "s.302", "IPC_302", "302 of the IPC", "120-B"
SECTION_RE = re.compile(r'\b(?:section|sec|s|sections|secs|§)\.?\s*[:\s\-]*([0-9]{1,4}(?:[A-Za-z\-]|\([0-9A-Za-z]+\))*)\b', flags=re.I)
IPC_PREFIX_RE = re.compile(r'\b(?:ipc|indian penal code)[\s_:,\-]*([0-9]{1,4}(?:[A-Za-z\-\(\)]*)?)\b', flags=re.I)
IPC_SIMPLE_NUM = re.compile(r'\b([0-9]{2,4})(?:\b|$)')

def regex_find_ids(text):
    hits = set()
    for m in SECTION_RE.finditer(text):
        tok = m.group(1)
        # normalize to IPC_### form if statute exists with that number
        digits = re.search(r'(\d+)', tok)
        if digits:
            cand = "IPC_" + digits.group(1)
            if cand in statute_ids:
                hits.add(cand)
    for m in IPC_PREFIX_RE.finditer(text):
        digits = re.search(r'(\d+)', m.group(1))
        if digits:
            cand = "IPC_" + digits.group(1)
            if cand in statute_ids:
                hits.add(cand)
    # also try bare numbers but avoid noise: require "Section" context in earlier matches mostly
    return hits

# ----------------- Load LegalBERT for per-label probabilities -----------------
print("Loading LegalBERT (for bert_prob features) from:", LEGALBERT_DIR)
tokenizer = AutoTokenizer.from_pretrained(LEGALBERT_DIR)
legalbert = AutoModelForSequenceClassification.from_pretrained(LEGALBERT_DIR)
legalbert.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
legalbert.to(device)

# we need label order used during training -> load label_list.json
label_list_path = Path(LEGALBERT_DIR)/"label_list.json"
if not label_list_path.exists():
    # try mlb.pkl if present (older pipeline)
    raise FileNotFoundError("label_list.json not found in LEGALBERT_DIR. Ensure you saved mlb.classes_ during training.")
with open(label_list_path,"r",encoding="utf-8") as f:
    label_list = json.load(f)
label_to_index = {lab:i for i,lab in enumerate(label_list)}
num_labels = len(label_list)
print("LegalBERT num_labels:", num_labels)

# batched inference helper: returns ndarray shape (N, num_labels) with probabilities
def legalbert_predict_probs(texts, batch_size=16, max_len=256):
    all_probs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
        enc = {k:v.to(device) for k,v in enc.items()}
        with torch.no_grad():
            out = legalbert(**enc)
            logits = out.logits.cpu().numpy()
            probs = 1/(1+np.exp(-logits))  # sigmoid
            all_probs.append(probs)
    return np.vstack(all_probs)

# ----------------- Candidate generation & feature extraction (build meta-training table) -----------------
def generate_candidates_for_row(text, gold_set=None, topk=TOPK_RETRIEVE):
    """
    Returns list of candidate tuples: (stat_id, retrieval_score, sbert_sim, regex_flag)
    """
    candidates = []
    # 1) regex candidates
    regex_ids = regex_find_ids(text)
    for rid in regex_ids:
        candidates.append((rid, 1.0, 1.0, 1))  # high retrieval/sim for exact regex

    # 2) retrieval candidates
    retrieved = retrieve_candidates_by_text(text, topk=topk)
    for sid, sdef, rscore in retrieved:
        candidates.append((sid, float(rscore), float(rscore), 0))

    # 3) expand neighbors for any candidate from retrieval
    # combine unique preserving order
    seen = set(); uniq=[]
    for sid, rscore, sim, rflag in candidates:
        if sid not in seen:
            uniq.append((sid,rscore,sim,rflag)); seen.add(sid)
        # neighbors
        neighs = statute_neighbors(sid, topk=TOPK_NEIGHBORS)
        for nsid, ndef, nscore in neighs:
            if nsid not in seen:
                uniq.append((nsid, float(nscore), float(nscore), 0)); seen.add(nsid)
    return uniq

# Build meta rows for a dataframe (streaming)
def build_meta_examples(df, max_rows=None, verbose=True):
    """
    For each row in df, generate candidate rows (one row per candidate) and compute features.
    Returns pandas DataFrame with columns:
      text_index, statute_id, bert_prob, retrieval_score, sbert_sim, regex_flag, prior, label (1/0)
    """
    rows = []
    texts = df['text'].tolist()
    BATCH_LEGAL = 32  # legalbert batch for computing full vector per doc
    # Precompute legalbert probs per doc (to fetch bert_prob easily)
    if verbose: print("Computing LegalBERT document-level probabilities (batched)...")
    probs_all = legalbert_predict_probs(texts, batch_size=BATCH_LEGAL)
    if verbose: print("Done LegalBERT probs shape:", probs_all.shape)

    for i, text in enumerate(tqdm(texts[:max_rows] if max_rows else texts)):
        gold = set(df['labels_list'].iloc[i])
        candidates = generate_candidates_for_row(text, gold, topk=TOPK_RETRIEVE)
        # For each candidate build features
        for sid, rscore, ssim, rflag in candidates:
            feat = {}
            feat['text_index'] = i
            feat['statute_id'] = sid
            # bert_prob: use legalbert doc-level probs mapping to this statute if present in label_list
            if sid in label_to_index:
                feat['bert_prob'] = float(probs_all[i, label_to_index[sid]])
            else:
                feat['bert_prob'] = 0.0
            feat['retrieval_score'] = float(rscore)
            feat['sbert_sim'] = float(ssim)
            feat['regex_flag'] = int(rflag)
            feat['prior'] = float(prior_map.get(sid, 0.0))
            feat['label'] = 1 if sid in gold else 0
            rows.append(feat)
        # negative sampling: sample some additional negatives if you want balance (optional)
        # Here we do nothing; later sampling used when training
    meta_df = pd.DataFrame(rows)
    return meta_df

print("Building meta examples for training (this may take a few minutes)...")
meta_train = build_meta_examples(train_df, max_rows=None)
print("Meta train rows:", len(meta_train))
meta_train.to_csv(OUT_DIR/"meta_train_raw.csv", index=False)

# ----------------- Negative sampling & prepare balanced meta train set -----------------
# Keep all positives, and sample some negatives to achieve a balanced ratio
pos = meta_train[meta_train['label']==1]
neg = meta_train[meta_train['label']==0]

# sample negatives: up to NEGATIVE_SAMPLE_PER_POS * len(pos)
n_neg_needed = min(len(neg), NEGATIVE_SAMPLE_PER_POS * max(1, len(pos)))
neg_sampled = neg.sample(n=n_neg_needed, random_state=META_RANDOM_STATE) if n_neg_needed>0 else neg
meta_train_bal = pd.concat([pos, neg_sampled]).sample(frac=1.0, random_state=META_RANDOM_STATE).reset_index(drop=True)
print("Meta train balanced shape:", meta_train_bal.shape, "positives:", len(pos), "neg sampled:", len(neg_sampled))

# ----------------- Features & training -----------------
FEATURE_COLS = ['bert_prob','retrieval_score','sbert_sim','regex_flag','prior']
X = meta_train_bal[FEATURE_COLS].values
y = meta_train_bal['label'].values

# Train LightGBM
dtrain = lgb.Dataset(X, label=y)
print("Training LightGBM meta-classifier...")
meta_model = lgb.train(LIGHTGBM_PARAMS, dtrain, num_boost_round=200)
# save
meta_model.save_model(str(OUT_DIR/"meta_model.txt"))
print("Saved meta model to", OUT_DIR/"meta_model.txt")

# ----------------- Build meta validation set (from dev) and tune thresholds per-statute -----------------
print("Building meta examples for dev (validation) to tune thresholds...")
meta_dev = build_meta_examples(dev_df, max_rows=None)
meta_dev.to_csv(OUT_DIR/"meta_dev_raw.csv", index=False)
# compute probabilities
X_dev = meta_dev[FEATURE_COLS].values
meta_dev['prob'] = meta_model.predict(X_dev)
# per-statute threshold tuning
per_label_thresholds = {}
for sid, g in meta_dev.groupby('statute_id'):
    y_true = g['label'].values
    y_prob = g['prob'].values
    if len(np.unique(y_true))==1 and y_true.sum()==0:
        per_label_thresholds[sid] = 0.5
        continue
    best_t, best_f1 = 0.5, 0.0
    for t in np.linspace(0.05,0.95,19):
        y_pred = (y_prob >= t).astype(int)
        f = f1_score(y_true, y_pred, zero_division=0)
        if f > best_f1:
            best_f1, best_t = f, t
    per_label_thresholds[sid] = float(best_t)

# fallback default threshold
default_threshold = 0.5
print("Computed per-label thresholds for", len(per_label_thresholds), "statutes.")
# save thresholds
with open(OUT_DIR/"per_label_thresholds.json","w",encoding="utf-8") as f:
    json.dump(per_label_thresholds, f, indent=2)

# ----------------- Evaluate on dev at sample/document level -----------------
def meta_predict_for_doc_list(df):
    texts = df['text'].tolist()
    probs_all = legalbert_predict_probs(texts, batch_size=32)
    preds = []
    for i,text in enumerate(texts):
        candidates = generate_candidates_for_row(text)
        row_preds = []
        for sid, rscore, ssim, rflag in candidates:
            feat = [0]*len(FEATURE_COLS)
            # bert prob
            feat[0] = float(probs_all[i, label_to_index[sid]]) if sid in label_to_index else 0.0
            feat[1] = float(rscore)
            feat[2] = float(ssim)
            feat[3] = int(rflag)
            feat[4] = float(prior_map.get(sid,0.0))
            prob = meta_model.predict(np.array(feat).reshape(1,-1))[0]
            # apply per-label threshold if present else default
            thr = per_label_thresholds.get(sid, default_threshold)
            if prob >= thr:
                row_preds.append(sid)
        preds.append(row_preds)
    return preds

print("Evaluating meta-ensemble on dev...")
dev_preds = meta_predict_for_doc_list(dev_df)
# convert to binary matrix using label_list from legalbert training (mlb compatible)
def preds_to_matrix(preds_list, label_list):
    mat = np.zeros((len(preds_list), len(label_list)), dtype=int)
    idx = {l:i for i,l in enumerate(label_list)}
    for i,plist in enumerate(preds_list):
        for p in plist:
            if p in idx:
                mat[i, idx[p]] = 1
    return mat

y_dev_true = np.array([ [1 if lab in row else 0 for lab in label_list] for row in dev_df['labels_list'] ])
y_dev_pred = preds_to_matrix(dev_preds, label_list)

print("Dev micro-F1:", f1_score(y_dev_true, y_dev_pred, average='micro'))
print("Dev macro-F1:", f1_score(y_dev_true, y_dev_pred, average='macro'))
print("Sample classification report (dev):")
print(classification_report(y_dev_true, y_dev_pred, target_names=label_list, zero_division=0))

# ----------------- Final: predict on test set and save predictions ---------------
if test_df is not None:
    print("Running meta-ensemble on test set...")
    test_preds = meta_predict_for_doc_list(test_df)
    test_mat = preds_to_matrix(test_preds, label_list)
    y_test_true = np.array([ [1 if lab in row else 0 for lab in label_list] for row in test_df['labels_list'] ])
    print("Test micro-F1:", f1_score(y_test_true, test_mat, average='micro'))
    print("Test macro-F1:", f1_score(y_test_true, test_mat, average='macro'))
    out = test_df.copy()
    out['preds_meta'] = [';'.join(p) for p in test_preds]
    out.to_csv(OUT_DIR/"test_meta_predictions.csv", index=False)
    print("Saved test predictions to", OUT_DIR/"test_meta_predictions.csv")

print("All done — meta-ensemble artifacts saved at", OUT_DIR)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 47.6 MB/s eta 0:00:00
Loaded datasets: train 42750 dev 10181 test 13019
Statutes loaded: 455
Loading SBERT: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

# Task
Modify `load_csv` in cell `RcJi-6NIN5ok` to ensure the DataFrame's index is reset to a unique, default index immediately after loading and renaming columns, but before selecting the final 'text' and 'labels' columns, to prevent `ValueError: cannot reindex on an axis with duplicate labels`.

## Modify load_csv in cell RcJi-6NIN5ok to ensure unique DataFrame index

### Subtask:
Adjust the `load_csv` function to apply `reset_index(drop=True)` to the DataFrame after initial loading and renaming, but before selecting the final columns. This ensures the returned DataFrame always has a unique, default index, preventing reindexing errors.


## Summary:

### Data Analysis Key Findings
*   The `load_csv` function was modified to prevent `ValueError: cannot reindex on an axis with duplicate labels` by ensuring the DataFrame always has a unique, default index.
*   This was achieved by applying `reset_index(drop=True)` to the DataFrame after loading and renaming columns, but before selecting the final 'text' and 'labels' columns.

### Insights or Next Steps
*   This change enhances the robustness of data loading by preventing potential errors related to non-unique indices in subsequent DataFrame operations.
*   The corrected `load_csv` function ensures consistency and reliability when processing CSV data, particularly for operations that depend on unique indexing.
